In [25]:
import pandas as pd
import numpy as np
import csv
import json
import os
import random
import time


In [26]:
 def diff(Rule1, Rule2):
    if Rule1[-1] != Rule2[-1]:
        return -1
    m = len(Rule1)
    count = 0
    for i in range(m-1):
        if Rule1[i] == Rule2[i]:
            count += 1
    return count/m

In [27]:
def sampling(base, ran, e, k = 2):
    num = len(base)
    R = []
    list_index = []
    while (len(R)<num*ran/100):
        index = random.randrange(num)
        while (index in list_index):
            index = random.randrange(num)
        T = []
        T.append(index)
        for i in range(index-k,index+k+1):
            if i < num:
                if i in list_index:
                    continue
                else:
                    if diff(base[i],base[index]) < 1 - e:
                        T.append(i)
        for i in T:
            temp = 0
            for j in range(len(R)):
                if diff(base[i],R[j]) < 1 - e:
                    continue
                else:
                    temp = 1
            if temp:
                T.remove(i)
        for i in T:
            R.append(base[i])
            list_index.append(i)

    return R

In [28]:
def combination(k, n):
    if k == 0 or k == n:
        return 1
    if k == 1:
        return n
    return combination(k - 1, n - 1) + combination(k, n - 1)

In [29]:
def calculateA(base):
    colum = len(base[0])
    row = len(base)
    A = np.zeros((row, combination(4, colum - 1)))

    for r1 in range(row):
        k = [0] * combination(4, colum - 1)
        temp = 0
        for a in range(0, colum - 4):
            for b in range(a + 1, colum - 3):
                for c in range(b + 1, colum - 2):
                    for d in range(c + 1, colum - 1):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r1][d] == base[r2][d]:
                                k[temp] += 1

                        A[r1][temp] = k[temp] / row
                        temp += 1
    print("done A")
    return A

In [30]:
def calculateM(base):
    colum = len(base[0])
    row = len(base)
    M = np.zeros((row, colum - 1))
    for t1 in range(row):
        k = [0] * (colum - 1)
        temp = 0
        for i in range(colum - 1):
            for t2 in range(row):
                if base[t1][i] == base[t2][i] and base[t1][colum - 1] == base[t2][colum - 1]:
                    k[temp] += 1
            M[t1][temp] = k[temp] / row
            temp += 1
    return M

In [31]:
def calculateB(base, A, M):
    colum = len(base[0])
    row = len(base)
    B = np.zeros((row, combination(3, colum - 1)))

    for r in range(row):
        temp = 0
        for a in range(0, colum - 3):
            for b in range(a + 1, colum - 2):
                for c in range(b + 1, colum - 1):
                    B[r][temp] = sum(A[r]) * min(M[r][a], M[r][b], M[r][c])
                    temp += 1
    print("done B")
    return B


In [32]:
 def calculateC(base, B):
    colum = len(base[0])
    row = len(base)
    cols = 6 * combination(3, colum - 1)
    C = np.zeros((row, cols))

    for r1 in range(row):
        temp = 0
        for i in range(1,7):
            for a in range(0, (colum - 3)):
                for b in range(a + 1, (colum - 2)):
                    for c in range(b + 1, (colum - 1)):
                        for r2 in range(row):
                            if base[r1][a] == base[r2][a] and base[r1][b] == base[r2][b] and base[r1][c] == base[r2][c] and base[r2][colum - 1] == i:
                                C[r1][temp] += B[r2][temp % combination(3, colum - 1)]
                        temp += 1
    print("done C")
    return C

In [33]:
def FISA(base, C, list):
    colum = len(base[0])
    row = len(base)

    cols = combination(3, (colum - 1))
    C0 = [0] * cols
    C1 = [0] * cols
    C2 = [0] * cols
    C3 = [0] * cols
    C4 = [0] * cols
    C5 = [0] * cols

    t = 0
    for a in range(0, colum - 3):
        for b in range(a + 1, colum - 2):
            for c in range(b + 1, colum - 1):
                for r in range(row-1):
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 1:
                        C0[t] = C[r][t + 0 * cols]
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 2:
                        C1[t] = C[r][t + 1 * cols]
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 3:
                        C2[t] = C[r][t + 2 * cols]
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 4:
                        C3[t] = C[r][t + 3 * cols]
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 5:
                        C4[t] = C[r][t + 4 * cols]
                    if base[r][a] == list[a] and base[r][b] == list[b] and base[r][c] == list[c] and base[r][colum-1] == 6:
                        C5[t] = C[r][t + 5 * cols]
                t += 1

    D0 = max(C0) + min(C0)
    D1 = max(C1) + min(C1)
    D2 = max(C2) + min(C2)
    D3 = max(C3) + min(C3)
    D4 = max(C4) + min(C4)
    D5 = max(C5) + min(C5)
    
    DARR = [D0,D1,D2,D3,D4,D5]
    
    return DARR.index(max(DARR)) + 1, max(DARR)/sum(DARR)

In [34]:
def Acc(A,B):
    result = 0
    B = [int(float(x)) for x in B]

    for i in range(len(A)):
        if int(A[i]) - int(B[i]) == 0:
            result += 1

    return round(result*100/len(A), 2)


def Tprecision(Pre, Act):
    TP = {}
    FP = {}
    unique_labels = set(Act) | set(Pre)
    for label in unique_labels:
        TP[label] = 0
        FP[label] = 0

    for i in range(len(Pre)):
        pred_label = int(Pre[i])
        true_label = int(Act[i])

        if pred_label == true_label:
            TP[pred_label] += 1
        else:
            FP[pred_label] += 1

    label_precision = {}
    for label in unique_labels:
        if TP[label] + FP[label] > 0:
            label_precision[label] = round(100 * TP[label] / (TP[label] + FP[label]), 2)
        else:
            label_precision[label] = 0
    print(label_precision)
    return label_precision


def Trecall(Pre, Act):
    TP = {}
    FN = {}
    unique_labels = set(Act) | set(Pre)

    for label in unique_labels:
        TP[label] = 0
        FN[label] = 0

    for i in range(len(Pre)):
        pred_label = int(Pre[i])
        true_label = int(Act[i])

        if pred_label == true_label:
            TP[true_label] += 1
        else:
            FN[true_label] += 1

    label_recall = {}
    for label in unique_labels:
        if TP[label] + FN[label] > 0:
            label_recall[label] = round(100 * TP[label] / (TP[label] + FN[label]), 2)
        else:
            label_recall[label] = 0

    return label_recall


def testAccuracy(base,Te,C):
    test = Te
    X = np.zeros(len(test))
    ddd = np.zeros(len(test))
    X_test = np.array(test).T[-1]
    for i in range(len(test)):
        X[i], ddd[i] = fs.FISA(base, C, test[i])
        listRank.append(ddd[i])
       
    res.append(X)
    listAcc.append(Acc(X,X_test))
    listPre.extend(list(Tprecision(X, X_test).values()))
    listRe.extend(list(Trecall(X, X_test).values()))



In [35]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('/kaggle/input/dataset-process-rule/Only Image Feature/Rule_List.csv')
df = df.values
df = [[int(float(x)) for x in row] for row in df]
traindf = [row[:] for row in df]

testdf = pd.read_csv("/kaggle/input/dataset-process-rule/Only Image Feature/FRB/TestDataRule.csv")
testdf = testdf.values
testdf = [[int(float(x)) for x in row] for row in testdf]
test = testdf

sampling_time = []
train_time = []
test_time = []
listAcc = []
listPre = []
listRe = []
timeTest = []
timeUpdate = []
listRank = []
res = []


def min_max_normalize(C):
    C_min = np.min(C, axis=0)
    C_max = np.max(C, axis=0)
    C_normalized = (C - C_min) / (C_max - C_min)
    return C_normalized

In [36]:
import sys
sys.path.append("/kaggle/input/module-cpp")
import fisa_module as fs
import time
for i in range(5):
    print(f"--------------------------------------Turn {i}---------------------------------")
    start = time.time()
    base = sampling(ran=20, base=traindf, e=0.2)
    totalTime = time.time() - start
    sampling_time.append(totalTime)
    print(f'len base {i}:', len(base))


    start = time.time()
    A = fs.calculateA(base)
    M = fs.calculateM(base)
    B = fs.calculateB(base,A,M)
    C = fs.calculateC(base,B)
    C_normal = min_max_normalize(C)
    totalTime = time.time() - start
    train_time.append(totalTime)


    start = time.time()
    testAccuracy(base,test,C_normal)
    totalTime = time.time() - start
    test_time.append(totalTime)
    



--------------------------------------Turn 0---------------------------------
len base 0: 114
{1: 35.51, 2: 33.33, 3: 100.0, 4: 100.0, 5: 100.0, 6: 100.0}
--------------------------------------Turn 1---------------------------------
len base 1: 116
{1: 35.97, 2: 40.0, 3: 88.24, 4: 100.0, 5: 100.0, 6: 100.0}
--------------------------------------Turn 2---------------------------------
len base 2: 115
{1: 38.46, 2: 40.0, 3: 100.0, 4: 85.71, 5: 100.0, 6: 100.0}
--------------------------------------Turn 3---------------------------------
len base 3: 115
{1: 35.61, 2: 30.0, 3: 83.33, 4: 66.67, 5: 100.0, 6: 100.0}
--------------------------------------Turn 4---------------------------------
len base 4: 113
{1: 34.06, 2: 45.45, 3: 100.0, 4: 100.0, 5: 100.0, 6: 0}


In [37]:
print('sampling_time:', sampling_time)
print('train_time:', train_time)
print('test_time:', test_time)
print('accuracy:', listAcc)
print('precision:',sum(listPre) / len(listPre) if listPre else 0)
print('recall:',sum(listRe) / len(listRe) if listRe else 0)

sampling_time: [0.0035774707794189453, 0.0038840770721435547, 0.0039103031158447266, 0.004270076751708984, 0.0038328170776367188]
train_time: [0.3644893169403076, 0.37882375717163086, 0.37105870246887207, 0.3855612277984619, 0.3556687831878662]
test_time: [4.190901041030884, 4.115832328796387, 4.06221342086792, 4.051751613616943, 3.9747824668884277]
accuracy: [45.51, 43.71, 49.7, 43.11, 41.92]
precision: 73.078
recall: 34.90533333333333
